In [1]:
# conda install -c rapidsai -c nvidia -c conda-forge dask-cuda cudatoolkit=11.6
# nvidia-smi
from sklearn.model_selection import train_test_split, KFold
import dask.dataframe as dd
import dask.multiprocessing
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os
from dask import compute
from sklearn.metrics import f1_score
# f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
import tensorflow as tf

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# from dask.distributed import Client, LocalCluster
# # client.close()
# cluster = LocalCluster()
# client = Client(cluster)
# print(cluster,client)
# client.close()
# http://localhost:8787/status

# from dask_cuda import LocalCUDACluster
# from dask.distributed import Client
# client.close()
# # http://127.0.0.1:34497/status
# cluster = LocalCUDACluster()
# client = Client(cluster)
# print(cluster,client)

2022-01-28 16:08:31.206393: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-01-28 16:08:32.861327: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-28 16:08:32.871195: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-28 16:08:33.145059: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-28 16:08:33.145109: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 Laptop GPU computeCapability: 8.6
coreClock: 1.282GHz coreCount: 30 deviceMemorySize: 6.00GiB deviceMemoryBandwidth: 268.26GiB/s
2022-01-28 16:08:33.145156: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-28 16:08:33.154568: I tensorflow/stream_exec

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1664328743793847039,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4933605152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6912303323531783733
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"]

In [3]:
#making_label_set

crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'0':'정상','1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = "정상"
    for disease_code,value in value.items():
        label = f'{disease_code}'
        disease_label_description[label] = f'{value}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{value}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [4]:
crop_label_description

{'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}

In [5]:
risk_label_description

{'0': '정상', '1': '초기', '2': '중기', '3': '말기'}

In [6]:
disease_label_description

{'00': '정상',
 'a1': '딸기잿빛곰팡이병',
 'a2': '딸기흰가루병',
 'b1': '냉해피해',
 'b6': '다량원소결핍 (N)',
 'b7': '다량원소결핍 (P)',
 'b8': '다량원소결핍 (K)',
 'a5': '토마토흰가루병',
 'a6': '토마토잿빛곰팡이병',
 'b2': '열과',
 'b3': '칼슘결핍',
 'a9': '파프리카흰가루병',
 'a10': '파프리카잘록병',
 'a3': '오이노균병',
 'a4': '오이흰가루병',
 'a7': '고추탄저병',
 'a8': '고추흰가루병',
 'a11': '시설포도탄저병',
 'a12': '시설포도노균병',
 'b4': '일소피해',
 'b5': '축과병'}

In [7]:
label_description

{'1_00_0': '딸기_정상',
 '1_a1_0': '딸기_딸기잿빛곰팡이병_정상',
 '1_a1_1': '딸기_딸기잿빛곰팡이병_초기',
 '1_a1_2': '딸기_딸기잿빛곰팡이병_중기',
 '1_a1_3': '딸기_딸기잿빛곰팡이병_말기',
 '1_a2_0': '딸기_딸기흰가루병_정상',
 '1_a2_1': '딸기_딸기흰가루병_초기',
 '1_a2_2': '딸기_딸기흰가루병_중기',
 '1_a2_3': '딸기_딸기흰가루병_말기',
 '1_b1_0': '딸기_냉해피해_정상',
 '1_b1_1': '딸기_냉해피해_초기',
 '1_b1_2': '딸기_냉해피해_중기',
 '1_b1_3': '딸기_냉해피해_말기',
 '1_b6_0': '딸기_다량원소결핍 (N)_정상',
 '1_b6_1': '딸기_다량원소결핍 (N)_초기',
 '1_b6_2': '딸기_다량원소결핍 (N)_중기',
 '1_b6_3': '딸기_다량원소결핍 (N)_말기',
 '1_b7_0': '딸기_다량원소결핍 (P)_정상',
 '1_b7_1': '딸기_다량원소결핍 (P)_초기',
 '1_b7_2': '딸기_다량원소결핍 (P)_중기',
 '1_b7_3': '딸기_다량원소결핍 (P)_말기',
 '1_b8_0': '딸기_다량원소결핍 (K)_정상',
 '1_b8_1': '딸기_다량원소결핍 (K)_초기',
 '1_b8_2': '딸기_다량원소결핍 (K)_중기',
 '1_b8_3': '딸기_다량원소결핍 (K)_말기',
 '2_00_0': '토마토_정상',
 '2_a5_0': '토마토_토마토흰가루병_정상',
 '2_a5_1': '토마토_토마토흰가루병_초기',
 '2_a5_2': '토마토_토마토흰가루병_중기',
 '2_a5_3': '토마토_토마토흰가루병_말기',
 '2_a6_0': '토마토_토마토잿빛곰팡이병_정상',
 '2_a6_1': '토마토_토마토잿빛곰팡이병_초기',
 '2_a6_2': '토마토_토마토잿빛곰팡이병_중기',
 '2_a6_3': '토마토_토마토잿빛곰팡이병_말기',
 '2_b2_0': '토마토_열과_정상',

In [8]:
@delayed
def label_encoding(label):
    global ensemble_label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_crop(label):
    global crop_label_encoder
    encoded_label = crop_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_disease(label):
    global disease_label_encoder
    encoded_label = disease_label_encoder[label]
    return encoded_label

@delayed
def label_encoding_risk(label):
    global risk_label_encoder
    encoded_label = risk_label_encoder[label]
    return encoded_label

def getlable(jsonpath,type):
    if type == "ensemble":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label

    elif type == "crop":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        crop = json_file['annotations']['crop']
        label = f'{crop}'
        return label

    elif type == "dc":
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        disease = json_file['annotations']['disease']
        label = f'{disease}'
        return label

    elif type == "risk":  
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)
        risk = json_file['annotations']['risk']
        label = f'{risk}'
        return label

    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    labelarr_crop = np.array([])
    labelarr_dc = np.array([])
    labelarr_risk = np.array([])


    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path,"ensemble")))
        labelarr = np.append(labelarr,label)

        label_crop = np.array(label_encoding_crop(getlable(json_path,"crop")))
        labelarr_crop = np.append(labelarr_crop,label_crop)

        label_dc = np.array(label_encoding_disease(getlable(json_path,"dc")))
        labelarr_dc = np.append(labelarr_dc,label_dc)

        label_risk = np.array(label_encoding_risk(getlable(json_path,"risk")))
        labelarr_risk = np.append(labelarr_risk,label_risk)


    return labelarr, labelarr_crop, labelarr_dc, labelarr_risk

In [9]:
@delayed
def get_all_image(imgpath_list,image_size):
    imgarr = np.empty((0,image_size,image_size,3), float)
    for ind,img_path in enumerate(imgpath_list):
        imgarr = delayed(np.append)(imgarr,img, axis = 0)
    return imgarr

In [10]:
%%time
label_list = sorted(glob(os.path.join(path,"data","train","*","*.json")))
y_train,y_train_crop,y_train_dc,y_train_risk, = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)
results_crop = dask.compute(*y_train_crop)
label_crop = np.array(results_crop)
#
results_dc = dask.compute(*y_train_dc)
label_disease = np.array(results_dc)
#
results_risk = dask.compute(*y_train_risk)
label_risk = np.array(results_risk)
label.shape,label_crop.shape,label_disease.shape,label_risk.shape

CPU times: user 4.1 s, sys: 312 ms, total: 4.41 s
Wall time: 5.18 s


((5767,), (5767,), (5767,), (5767,))

In [11]:
%%time
@delayed
def imageresize(img):
    # img = dask.delayed(cv2.resize)(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    # img = dask.delayed(img.astype(np.float32)/255)  ##픽셀값을 0~1사이로 정규화
    # # img = np.transpose(img, (2,0,1))
    # return dask.delayed(img.reshape)(-1,224,224,3)
    img = cv2.resize(img, dsize=(100, 100), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
    # img = np.transpose(img, (2,0,1))
    return da.array(img)
#making_img_set
row_img = dask_image.imread.imread(os.path.join(path,"data","train","*","*.jpg"))
train = [imageresize(img) for img in row_img]
train_x = da.array(dask.compute(*train))

CPU times: user 44.4 s, sys: 3.91 s, total: 48.3 s
Wall time: 16.1 s


In [12]:
train_x = train_x.reshape(5767,-1)
label = da.array(label)

## 100열

In [27]:
X, X_test, y, y_test = train_test_split(train_x, label, test_size=0.2, shuffle= True,stratify=label)
X, X_test, y, y_test = X.rechunk((1000,-1))[:,:100], X_test.rechunk((1000,-1))[:,:100], y.rechunk((1000,-1)), y_test.rechunk((1000,-1))

In [28]:
X

dask.array<getitem, shape=(4613, 100), dtype=float32, chunksize=(1000, 100), chunktype=numpy.ndarray>

In [37]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 1min 48s, sys: 460 ms, total: 1min 49s
Wall time: 10.5 s


In [38]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 1min 49s, sys: 350 ms, total: 1min 49s
Wall time: 8.67 s


In [39]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 2min 17s, sys: 450 ms, total: 2min 17s
Wall time: 10.6 s


In [40]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 2min 2s, sys: 410 ms, total: 2min 2s
Wall time: 11.3 s


## 1000열

In [41]:
X, X_test, y, y_test = train_test_split(train_x, label, test_size=0.2, shuffle= True,stratify=label)
X, X_test, y, y_test = X.rechunk((1000,-1))[:,:1000], X_test.rechunk((1000,-1))[:,:1000], y.rechunk((1000,-1)), y_test.rechunk((1000,-1))

In [42]:
X

dask.array<getitem, shape=(4613, 1000), dtype=float32, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [43]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 10min 45s, sys: 639 ms, total: 10min 46s
Wall time: 42.3 s


In [44]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 10min 43s, sys: 439 ms, total: 10min 43s
Wall time: 42 s


In [45]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 11min 1s, sys: 379 ms, total: 11min 1s
Wall time: 43.1 s


In [46]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 10min 22s, sys: 429 ms, total: 10min 22s
Wall time: 42.6 s


## 1000열 3000청크사이즈

In [63]:
X, X_test, y, y_test = train_test_split(train_x, label, test_size=0.2, shuffle= True,stratify=label)
X, X_test, y, y_test = X.rechunk((3000,-1))[:,:1000], X_test.rechunk((3000,-1))[:,:1000], y.rechunk((3000,-1)), y_test.rechunk((3000,-1))

In [64]:
X

dask.array<getitem, shape=(4613, 1000), dtype=float32, chunksize=(3000, 1000), chunktype=numpy.ndarray>

In [65]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 12min 53s, sys: 810 ms, total: 12min 54s
Wall time: 51.1 s


In [66]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.254768059852604

In [67]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 10min 54s, sys: 570 ms, total: 10min 55s
Wall time: 43.1 s


In [68]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.254768059852604

In [69]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 11min 24s, sys: 580 ms, total: 11min 25s
Wall time: 46.6 s


In [70]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.254768059852604

In [ ]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

## 1000열 넘파이

In [55]:
X, X_test, y, y_test = train_test_split(train_x, label, test_size=0.2, shuffle= True,stratify=label)
X, X_test, y, y_test = np.array(X)[:,:1000],np.array(X_test)[:,:1000],np.array(y),np.array(y_test),

In [56]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 10min 47s, sys: 220 ms, total: 10min 47s
Wall time: 40.8 s


In [57]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 10min 34s, sys: 69.5 ms, total: 10min 34s
Wall time: 39.9 s


In [58]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 10min 56s, sys: 189 ms, total: 10min 56s
Wall time: 41.3 s


In [59]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred1 = XGB_mod1.predict(X_test)

CPU times: user 10min 47s, sys: 190 ms, total: 10min 47s
Wall time: 40.7 s


## cluster 테스트

In [60]:
X, X_test, y, y_test = train_test_split(train_x, label, test_size=0.2, shuffle= True,stratify=label)
X, X_test, y, y_test = X.rechunk((3000,-1))[:,:1000], X_test.rechunk((3000,-1))[:,:1000], y.rechunk((3000,-1)), y_test.rechunk((3000,-1))

In [ ]:
%%time
from dask.distributed import Client, LocalCluster
#http://localhost:3258/status
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=4,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

<Client: 'tcp://127.0.0.1:42121' processes=4 threads=16, memory=24.59 GiB>


[17:20:18] task [xgboost.dask]:tcp://127.0.0.1:46113 got new rank 0
[17:20:18] task [xgboost.dask]:tcp://127.0.0.1:39685 got new rank 1


[0]	train-mlogloss:2.93766
[1]	train-mlogloss:1.99399
[2]	train-mlogloss:1.58934
[3]	train-mlogloss:1.29963
34.996373653411865
CPU times: user 11.4 s, sys: 1.52 s, total: 12.9 s
Wall time: 38.4 s


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

In [34]:
%%time
from dask.distributed import Client, LocalCluster
#http://localhost:3258/status
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "gpu_hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=4,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

<Client: 'tcp://127.0.0.1:46851' processes=4 threads=16, memory=24.59 GiB>


[16:35:21] task [xgboost.dask]:tcp://127.0.0.1:45085 got new rank 0
[16:35:21] task [xgboost.dask]:tcp://127.0.0.1:45317 got new rank 1
[16:35:21] task [xgboost.dask]:tcp://127.0.0.1:41293 got new rank 2
[16:35:21] task [xgboost.dask]:tcp://127.0.0.1:35087 got new rank 3
distributed.worker - WARNING - Compute Failed
Function:  dispatched_train
args:      ('tcp://127.0.0.1:45317', [b'DMLC_NUM_WORKER=4', b'DMLC_TRACKER_URI=172.24.219.36', b'DMLC_TRACKER_PORT=9097', b'DMLC_TASK_ID=[xgboost.dask]:tcp://127.0.0.1:45317'], {'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_categorical': False, 'parts': [(array([[0.14509805, 0.34117648, 0.09411765, ..., 0.1764706 , 0.08627451,
        0.19607843],
       [0.3529412 , 0.41568628, 0.20784314, ..., 0.35686275, 0.13333334,
        0.19215687],
       [0.34117648, 0.28627452, 0.25490198, ..., 0.48235294, 0.11764706,
        0.36078432],
       ...,
       [0.04313726, 0.047058

XGBoostError: [16:36:17] ../src/tree/updater_gpu_hist.cu:770: Exception in gpu_hist: [16:36:17] ../src/common/device_helpers.cu:61: Check failed: n_uniques == world (1 vs. 4) : Multiple processes within communication group running on same CUDA device is not supported. 73bc4708914a6a63232a2564cb70ed45

Stack trace:
  [bt] (0) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31ddcd) [0x7ff3acb89dcd]
  [bt] (1) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31ebf5) [0x7ff3acb8abf5]
  [bt] (2) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4d7f42) [0x7ff3acd43f42]
  [bt] (3) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4e1d26) [0x7ff3acd4dd26]
  [bt] (4) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x17d683) [0x7ff3ac9e9683]
  [bt] (5) /home/go/miniconda3/envs/venv_01/lib/python

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

In [62]:
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCUDACluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=4,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/dask_cuda/utils.py:216: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  % device_index
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


<Client: 'tcp://127.0.0.1:43351' processes=1 threads=1, memory=24.59 GiB>


[17:21:02] task [xgboost.dask]:tcp://127.0.0.1:37983 got new rank 0


[0]	train-mlogloss:2.93766
[1]	train-mlogloss:1.99399
[2]	train-mlogloss:1.58934
[3]	train-mlogloss:1.29963
31.139005661010742


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

In [35]:
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCUDACluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "gpu_hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=4,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            # y_pred = xgb.dask.predict(client,out,X_test).compute()
            

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/dask_cuda/utils.py:216: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  % device_index
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


<Client: 'tcp://127.0.0.1:33583' processes=1 threads=1, memory=24.59 GiB>


[16:36:51] task [xgboost.dask]:tcp://127.0.0.1:42903 got new rank 0
distributed.worker - WARNING - Compute Failed
Function:  dispatched_train
args:      ('tcp://127.0.0.1:42903', [b'DMLC_NUM_WORKER=1', b'DMLC_TRACKER_URI=172.24.219.36', b'DMLC_TRACKER_PORT=9097', b'DMLC_TASK_ID=[xgboost.dask]:tcp://127.0.0.1:42903'], {'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_categorical': False, 'parts': [(array([[0.18431373, 0.15686275, 0.14509805, ..., 0.07843138, 0.05098039,
        0.10980392],
       [0.1254902 , 0.25490198, 0.16078432, ..., 0.77254903, 0.6431373 ,
        0.5803922 ],
       [0.5294118 , 0.7647059 , 0.4117647 , ..., 0.44705883, 0.11764706,
        0.22745098],
       ...,
       [0.30588236, 0.47843137, 0.28235295, ..., 0.72156864, 0.24313726,
        0.4509804 ],
       [0.4392157 , 0.3019608 , 0.20392157, ..., 0.42352942, 0.1764706 ,
        0.39215687],
       [0.15686275, 0.22352941, 0.10980392, 

XGBoostError: [16:36:51] ../src/tree/updater_gpu_hist.cu:770: Exception in gpu_hist: [16:36:51] ../src/common/device_helpers.cuh:132: NCCL failure :unhandled system error ../src/common/device_helpers.cu(67)
Stack trace:
  [bt] (0) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31ddcd) [0x7f941a872dcd]
  [bt] (1) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x320f29) [0x7f941a875f29]
  [bt] (2) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31f12a) [0x7f941a87412a]
  [bt] (3) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4d7f42) [0x7f941aa2cf42]
  [bt] (4) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4e1d26) [0x7f941aa36d26]
  [bt] (5) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x17d683) [0x7f941a6d2683]
  [bt] (6) /home/go/mini

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

In [36]:
xgb.__version__

'1.5.2'

## 데이터 전처리 속도 비교

In [89]:
%%time
#making_csv_set
row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
       '내부 이슬점 최저': 'object',
       '내부 이슬점 평균': 'object'})
partitions = row_csv.to_delayed()
# datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
X_train = da.array(dask.compute(*datas))


CPU times: user 3min 35s, sys: 21 s, total: 3min 56s
Wall time: 3min 26s


In [90]:
%%time
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCUDACluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
                '내부 이슬점 최저': 'object',
                '내부 이슬점 평균': 'object'})
            partitions = row_csv.to_delayed()
            # datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
            datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
            X_train = da.array(dask.compute(*datas))

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/dask_cuda/utils.py:216: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  % device_index
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


CPU times: user 1min 20s, sys: 4.04 s, total: 1min 24s
Wall time: 4min 19s


In [91]:
%%time
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
                '내부 이슬점 최저': 'object',
                '내부 이슬점 평균': 'object'})
            partitions = row_csv.to_delayed()
            # datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
            datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
            X_train = da.array(dask.compute(*datas))

CPU times: user 1min 6s, sys: 2.6 s, total: 1min 9s
Wall time: 1min 57s


## csv셋 XGBClassifier

In [72]:
%%time
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
                '내부 이슬점 최저': 'object',
                '내부 이슬점 평균': 'object'})
            partitions = row_csv.to_delayed()
            # datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
            datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
            X_train = da.array(dask.compute(*datas))

In [73]:
X_train = X_train.reshape(5767,-1)

In [75]:
X_train

dask.array<reshape, shape=(5767, 2610), dtype=float64, chunksize=(5767, 2610), chunktype=numpy.ndarray>

In [76]:
X, X_test, y, y_test = train_test_split(X_train, label, test_size=0.2, shuffle= True,stratify=label)

In [81]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 11min 48s, sys: 578 ms, total: 11min 49s
Wall time: 44.8 s


In [82]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.8525408215969777

In [83]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 11min 15s, sys: 307 ms, total: 11min 15s
Wall time: 42.7 s


In [84]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.8525408215969777

In [85]:
%%time
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 13min 8s, sys: 658 ms, total: 13min 9s
Wall time: 50.6 s


In [86]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.8525408215969777

In [87]:
%%time
params = {"tree_method": "hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "single_precision_histogram" : True,
          }

start = time.time()
XGB_mod1 = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(X,y)
# print(time.time()-start)
y_pred = XGB_mod1.predict(X_test)

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/core.py:502: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  format(", ".join(args_msg)), FutureWarning
/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


CPU times: user 10min 16s, sys: 258 ms, total: 10min 16s
Wall time: 38.8 s


In [88]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.8525408215969777

## csv셋 xgb.dask, numboost 4

In [ ]:
%%time
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
                '내부 이슬점 최저': 'object',
                '내부 이슬점 평균': 'object'})
            partitions = row_csv.to_delayed()
            # datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
            datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
            X_train = da.array(dask.compute(*datas))

In [92]:
X_train = X_train.reshape(5767,-1)

In [ ]:
X_train

dask.array<reshape, shape=(5767, 2610), dtype=float64, chunksize=(5767, 2610), chunktype=numpy.ndarray>

In [94]:
X, X_test, y, y_test = train_test_split(X_train, label, test_size=0.2, shuffle= True,stratify=label)

In [95]:
%%time
from dask.distributed import Client, LocalCluster
#http://localhost:3258/status
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=4,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

<Client: 'tcp://127.0.0.1:43759' processes=4 threads=16, memory=24.59 GiB>


[17:49:06] task [xgboost.dask]:tcp://127.0.0.1:32875 got new rank 0


[0]	train-mlogloss:0.63510
[1]	train-mlogloss:0.36299
[2]	train-mlogloss:0.26461
[3]	train-mlogloss:0.20724
34.202969789505005
CPU times: user 3.09 s, sys: 729 ms, total: 3.82 s
Wall time: 36.2 s


In [96]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.7776970006305945

In [97]:
%%time
from dask.distributed import Client, LocalCluster
#http://localhost:3258/status
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "gpu_hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=4,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

<Client: 'tcp://127.0.0.1:35303' processes=4 threads=16, memory=24.59 GiB>


[17:49:43] task [xgboost.dask]:tcp://127.0.0.1:32783 got new rank 0
distributed.worker - WARNING - Compute Failed
Function:  dispatched_train
args:      ('tcp://127.0.0.1:32783', [b'DMLC_NUM_WORKER=1', b'DMLC_TRACKER_URI=172.24.219.36', b'DMLC_TRACKER_PORT=9097', b'DMLC_TASK_ID=[xgboost.dask]:tcp://127.0.0.1:32783'], {'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_categorical': False, 'parts': [(array([[27.8, 28. , 27.7, ...,  0. ,  0. ,  0. ],
       [15.4, 15.7, 15.1, ..., 10.8, 11. , 10.6],
       [27.4, 27.6, 27.2, ..., 24. , 24. , 24. ],
       ...,
       [29.4, 29.6, 29.3, ..., 19.5, 20.5, 19. ],
       [18.5, 18.6, 18.4, ..., 10.1, 10.2,  9.9],
       [13.4, 13.5, 13.3, ..., 10.6, 10.7, 10.6]]), array([104, 126, 129, ...,  54,  68, 122]))], 'is_quantile': False}, 140692356424400, [({'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_c

XGBoostError: [17:49:43] ../src/tree/updater_gpu_hist.cu:770: Exception in gpu_hist: [17:49:43] ../src/common/device_helpers.cuh:132: NCCL failure :unhandled system error ../src/common/device_helpers.cu(67)
Stack trace:
  [bt] (0) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31ddcd) [0x7f8b7b034dcd]
  [bt] (1) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x320f29) [0x7f8b7b037f29]
  [bt] (2) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31f12a) [0x7f8b7b03612a]
  [bt] (3) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4d7f42) [0x7f8b7b1eef42]
  [bt] (4) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4e1d26) [0x7f8b7b1f8d26]
  [bt] (5) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x17d683) [0x7f8b7ae94683]
  [bt] (6) /home/go/mini

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

In [98]:
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCUDACluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=4,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/dask_cuda/utils.py:216: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  % device_index
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


<Client: 'tcp://127.0.0.1:41617' processes=1 threads=1, memory=24.59 GiB>


[17:49:57] task [xgboost.dask]:tcp://127.0.0.1:40155 got new rank 0


[0]	train-mlogloss:0.63510
[1]	train-mlogloss:0.36299
[2]	train-mlogloss:0.26461
[3]	train-mlogloss:0.20724
23.923319339752197


In [99]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.7776970006305945

## csv셋 xgb.dask, numboost 10

In [ ]:
%%time
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            row_csv = dd.read_csv(os.path.join(path,"data","train","*","*.csv"),dtype={'외부 누적일사 평균': 'object','내부 이슬점 최고': 'object',
                '내부 이슬점 최저': 'object',
                '내부 이슬점 평균': 'object'})
            partitions = row_csv.to_delayed()
            # datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features].values,290,9) for part in partitions]
            datas = [padding(part.drop_duplicates(subset=['측정시각'])[csv_features][part != "-"].dropna().values,290,9) for part in partitions[:]]
            X_train = da.array(dask.compute(*datas))

In [ ]:
X_train = X_train.reshape(5767,-1)

In [ ]:
X_train

dask.array<reshape, shape=(5767, 2610), dtype=float64, chunksize=(5767, 2610), chunktype=numpy.ndarray>

In [ ]:
X, X_test, y, y_test = train_test_split(X_train, label, test_size=0.2, shuffle= True,stratify=label)

In [100]:
%%time
from dask.distributed import Client, LocalCluster
#http://localhost:3258/status
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

<Client: 'tcp://127.0.0.1:34739' processes=4 threads=16, memory=24.59 GiB>


[17:51:02] task [xgboost.dask]:tcp://127.0.0.1:33983 got new rank 0


[0]	train-mlogloss:0.63510
[1]	train-mlogloss:0.36299
[2]	train-mlogloss:0.26461
[3]	train-mlogloss:0.20724
[4]	train-mlogloss:0.16719
[5]	train-mlogloss:0.14720


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

0.7776970006305945

In [ ]:
%%time
from dask.distributed import Client, LocalCluster
#http://localhost:3258/status
if __name__ == "__main__":
    with LocalCluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "gpu_hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

<Client: 'tcp://127.0.0.1:35303' processes=4 threads=16, memory=24.59 GiB>


[17:49:43] task [xgboost.dask]:tcp://127.0.0.1:32783 got new rank 0
distributed.worker - WARNING - Compute Failed
Function:  dispatched_train
args:      ('tcp://127.0.0.1:32783', [b'DMLC_NUM_WORKER=1', b'DMLC_TRACKER_URI=172.24.219.36', b'DMLC_TRACKER_PORT=9097', b'DMLC_TASK_ID=[xgboost.dask]:tcp://127.0.0.1:32783'], {'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_categorical': False, 'parts': [(array([[27.8, 28. , 27.7, ...,  0. ,  0. ,  0. ],
       [15.4, 15.7, 15.1, ..., 10.8, 11. , 10.6],
       [27.4, 27.6, 27.2, ..., 24. , 24. , 24. ],
       ...,
       [29.4, 29.6, 29.3, ..., 19.5, 20.5, 19. ],
       [18.5, 18.6, 18.4, ..., 10.1, 10.2,  9.9],
       [13.4, 13.5, 13.3, ..., 10.6, 10.7, 10.6]]), array([104, 126, 129, ...,  54,  68, 122]))], 'is_quantile': False}, 140692356424400, [({'feature_names': None, 'feature_types': None, 'feature_weights': None, 'meta_names': ['labels'], 'missing': None, 'enable_c

XGBoostError: [17:49:43] ../src/tree/updater_gpu_hist.cu:770: Exception in gpu_hist: [17:49:43] ../src/common/device_helpers.cuh:132: NCCL failure :unhandled system error ../src/common/device_helpers.cu(67)
Stack trace:
  [bt] (0) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31ddcd) [0x7f8b7b034dcd]
  [bt] (1) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x320f29) [0x7f8b7b037f29]
  [bt] (2) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x31f12a) [0x7f8b7b03612a]
  [bt] (3) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4d7f42) [0x7f8b7b1eef42]
  [bt] (4) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x4e1d26) [0x7f8b7b1f8d26]
  [bt] (5) /home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x17d683) [0x7f8b7ae94683]
  [bt] (6) /home/go/mini

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')

In [ ]:
from dask.distributed import Client, LocalCluster
from dask_cuda import LocalCUDACluster
#http://localhost:3258/status

if __name__ == "__main__":
    with LocalCUDACluster(dashboard_address=':3258') as cluster:
        with Client(cluster) as client:
            print(client)
            # X and y must be Dask dataframes or arrays
            params = {"verbosity": 0, 
            "tree_method": "hist", 
            "objective": 'multi:softmax',
            "num_class":len(label_description),
            }

           
            start = time.time()
            dtrain = xgb.dask.DaskDMatrix(client, X,y)
            out = xgb.dask.train(
                client,
                params,
                dtrain,
                num_boost_round=10,
                evals=[(dtrain, "train")],
            )
            print(time.time()-start)
            y_pred = xgb.dask.predict(client,out,X_test).compute()
            

/home/go/miniconda3/envs/venv_01/lib/python3.7/site-packages/dask_cuda/utils.py:216: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  % device_index
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


<Client: 'tcp://127.0.0.1:41617' processes=1 threads=1, memory=24.59 GiB>


[17:49:57] task [xgboost.dask]:tcp://127.0.0.1:40155 got new rank 0


[0]	train-mlogloss:0.63510
[1]	train-mlogloss:0.36299
[2]	train-mlogloss:0.26461


In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred,average='macro')